## Import Modules

In [1]:
import Dataset as ds
import config as cfg

import torch
from ultralytics import settings
from ultralytics import YOLO
from clearml import Task
import clearml
clearml.browser_login()
import gc


# Yolo Config

In [2]:
# Ultralytics settings
settings.reset()
settings.update({'datasets_dir': cfg.DATASET_DIR.replace('/','\\')})
print(settings)

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")

#Disable Data Augmentation (potentially)
#hsv_h: 0.0  # hue
#hsv_s: 0.0  # saturation
#hsv_v: 0.0  # value
#degrees: 0.0  # rotation
#translate: 0.0  # translate
#scale: 0.0  # scale
#shear: 0.0  # shear
#perspective: 0.0  # perspective
#flipud: 0.0  # flip up-down
#fliplr: 0.0  # flip left-right
#mosaic: 0.0  # mosaic
#mixup: 0.0  # mixup


{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets', 'weights_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True


## Load Dataset

In [3]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

#train_df.info()

In [4]:
# augmentation metadata

#augmentation_metadata = {
#    'methods': {
#        'hsv': {
#            'parameters': {
#                'type': 'h',  # Probability of applying the augmentation
#                'shift': 50,
#                'p': 1.0
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'flip': {
#            'parameters': {
#                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'rotate': {
#            'parameters': {
#                'angle': 90,  # Rotation angle
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 30% of the training images
#        },
#        'bnc': {
#            'parameters': {
#                'alpha': 1.5,  # contrast control
#                'beta': 50, # brightness control
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'histEq': {
#            'parameters': {
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'whiteBal': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'sharpen': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'gaussian': {
#            'parameters': {
#                'var_limit': (10.0, 50.0) # range of noise varaince
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'zoom': {
#            'parameters': {
#                'zoom_factor': 3  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        }       
#    }
#}


# Test Parameters

In [5]:
# name dataset to be create as well as task name
dataset_name = 'test500'

# project and task constant parameters
project_name= cfg.PROJECT_NAME
dataset_dir = cfg.DATASET_DIR
project_dir = f'{cfg.PROJECT_DIR}/{dataset_name}/'
class_names = cfg.YOLO_CLASS_NAMES

# task specifc training parameters
epochs = 20
model_variant = "yolov8n"

"""
Methods: 
 - flip
 - rotate
 - bnc
 - gaussian
 - histEq
 - whiteBal
 - sharpen
 - zoom
 - hsv
"""

augmentation_metadata = {
    'methods': {        
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}

# Create Datasets

In [6]:
from sklearn.model_selection import train_test_split

_, test_train_df = train_test_split(
    train_df,
    test_size=500,  # Number of items you want in your sample
    stratify=train_df['ac'],  # Stratify based on the combined column
    random_state=42  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    valid_df,
    test_size=100,  # Number of items you want in your sample
    stratify=valid_df['ac'],  # Stratify based on the combined column
    random_state=42  # Ensures reproducibility
)

# create sub dataset
ds.create_sub_dataset(dataset_name, test_train_df, test_val_df, class_names, dataset_dir)

# augment dataset
ds.augment_dataset(f'{dataset_dir}/{dataset_name}', augmentation_metadata)

Removing dataset if pre-existing
Copying training files:


Copying files:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying validation files:


Copying files:   0%|          | 0/200 [00:00<?, ?it/s]

Dataset 'test500' created at C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets


Removing existing directories/files:   0%|          | 0/3 [00:00<?, ?it/s]

Copying files:   0%|          | 0/500 [00:00<?, ?it/s]

Copying files:   0%|          | 0/500 [00:00<?, ?it/s]

Applying augmentations:   0%|          | 0/250 [00:00<?, ?it/s]

An error occurred: Expected y_min for bbox (0.29860800000000004, -0.02214949999999999, 0.8637140000000001, 0.3545875, 0) to be in the range [0.0, 1.0], got -0.02214949999999999.
An error occurred: Expected y_max for bbox (0.36195600000000006, 0.7324, 0.764684, 1.000886, 0) to be in the range [0.0, 1.0], got 1.000886.


# Training

In [7]:
# train on pure dataset

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}\\{dataset_name}.yaml'
project =  project_dir + 'pure' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{dataset_name}-pure"
)
task.set_parameter("model_variant", model_variant)

# Define Yolo model
model = YOLO(f'{model_variant}.yaml') # train on model which is not pre-trained
#model = YOLO(f'{model_variant}.pt')

#train args
args = dict(data=dataset_path, 
            epochs=epochs, 
            device=0, 
            close_mosaic=epochs, # disable mosaic augmentation
            seed=42)
task.connect(args)

# train model
results=model.train(**args, project=project)

# validate model
metrics = model.val()

# close task for next run
task.close()




ClearML Task: created new task id=a583680c46ad461fba678f8d69525bf5
2024-04-14 01:52:46,791 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/9e665058094a4ecd9f15a8a48b4a89d5/experiments/a583680c46ad461fba678f8d69525bf5/output/log
New https://pypi.org/project/ultralytics/8.1.47 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets\test500\test500.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/test500/pure, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=F

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 2105.00it/s]


train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 516.40it/s]

val: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid.cache


...


20 epochs completed in 0.036 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train9\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train9\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train9\weights\best.pt...
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.83it/s]


                   all        100        100      0.486       0.59       0.43      0.215
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train9
2024-04-14 01:55:53,739 - clearml.storage - INFO - Starting upload: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train9\weights\best.pt => https://files.clear.ml/Intruder-Aircraft-Detection/test500/test500-pure.a583680c46ad461fba678f8d69525bf5/models/best.pt
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.18it/s]


                   all        100        100       0.49        0.6      0.435      0.216
Speed: 0.6ms preprocess, 8.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train92


In [8]:
# train on augmented dataset

# clear variables
torch.cuda.empty_cache()
gc.collect()

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}\\{dataset_name}-aug.yaml'
project =  project_dir + 'augmented' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{dataset_name}-augmented"
)
task.set_parameter("model_variant", model_variant) 

# Define Yolo model
model = YOLO('yolov8n.yaml')
#model = YOLO('yolov8n.pt')

# train model
results = model.train(data=dataset_path, 
                      epochs=epochs, 
                      device=0, 
                      project=project,
                      close_mosaic=epochs, # disable mosaic augmentation
                      seed=42)

# validate model
metrics = model.val()

# close task for next/last run
task.close()



ClearML Task: created new task id=a0cec01fd3cb4fd18ae8ff2d2f471904
ClearML results page: https://app.clear.ml/projects/014a6c0f7a5b4a9eb1d008d5ee1516cc/experiments/a0cec01fd3cb4fd18ae8ff2d2f471904/output/log
New https://pypi.org/project/ultralytics/8.1.47 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets\test500\test500-aug.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/test500/augmented, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freez

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train-aug... 748 images, 0 backgrounds, 0 corrupt: 100%|██████████| 748/748 [00:00<00:00, 1156.66it/s]


train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train-aug.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]


Plotting labels to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train7
Starting training for 20 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.056 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train7\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train7\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train7\weights\best.pt...
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.87it/s]


                   all        100        100      0.588       0.53      0.511      0.243
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train7
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.21it/s]


                   all        100        100      0.575       0.52      0.502       0.24
Speed: 0.4ms preprocess, 3.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train72
